In [1]:

'''Question. 1
Covert below query to Pyspark code:

select City , count(City) as grouped_city, 
row_number() over (order by count(City) desc) activity_rank 
from table 
group by City;
'''

import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast

spark=SparkSession.builder.getOrCreate()

df=spark.read.format("csv").option("inferSchema", True).option("header", True).load("sample1.csv")
#df.show()


df_1=df.select("City").groupBy("City").agg(count("City").alias("grouped_city"))
#df_1.show()

window_df=Window.orderBy(col("grouped_city").desc())
df_final=df_1.withColumn("activity_rank", row_number().over(window_df))
df_final.show()

'''
Question 2:
Convert below query to pyspark code:

select user_id, round(sum(listen_duration)/60) as 'total_listening', count(distinct(song_id)) as 'distinct_songs'
from listening_habits
group by user_id;
'''

df_group=df.groupBy("user_id").agg(sum("listen_duration").alias("total_listening"),countDistinct("song_id").alias("distinct_songs"))
df_final=df_group.withColumn("total_listening", round(df_group.total_listening/60))
df_final.show()



'''
Question 3:
Convert below query to pyspark code:

with cte as
(
select *, lead(ID) over (order by ID) as next_id,
lag(ID) over (order by ID) as prev_id
from Students  
) 
select case
when ID%2!=0 and next_id is not null then next_id
when ID%2=0 then prev_id
when ID%2!=0 and next_id is null then ID
end
as ID, name
from cte;
'''

window_df=Window.orderBy(col('ID'))
df=spark.read.format("csv").option("inferSchema", True).option("header", True).load("example_student.csv")

result_df=df.withColumn("next_id", lead("ID").over(window_df)).withColumn("prev_id", lag("ID").over(window_df))

df1=result_df.withColumn("result_id", when((df.ID%2!=0) & (result_df.next_id.isNotNull()), result_df.next_id)
                                        .when(df.ID%2==0, result_df.prev_id)
                                        .when((df.ID%2!=0) & (result_df.next_id.isNull()), df.ID))

df1.select("Name", "result_id").show()


'''
Question 4:
Convert below query to pyspark code:

select employees.id, employees.name from
employees left join terminations
on employees.id=terminations.emp_id
where (terminations.term_date is null or terminations.term_date>'2016-01-02')
order by employees.hire_date;
'''

emp_df=spark.read.format("csv").option("header", True).option("inferSchema", True).load("emp1.csv")
term_df=spark.read.format("csv").option("header", True).option("inferSchema", True).load("term1.csv")
result_df=emp_df.join(term_df, emp_df.id==term_df.emp_id, "left").filter((col("term_date").isNull()) | (col("term_date")> '2016-01-02')).orderBy("hire_date")

result_df.select("id", "name").show()


'''
Question 5:
Convert below query into pyspark code:

select * from customers
where id not in 
(
select customers.id from customers inner join orders1
on customers.id = orders1.customerId
)
order by customers.name;
'''

cust_df=spark.read.format("csv").option("inferSchema", True).option("header", True).load("cust1.csv")
order_df=spark.read.format("csv").option("inferSchema", True).option("header", True).load("order1.csv")

join_df=cust_df.join(order_df, cust_df.id==order_df.customerId, "inner")

cust_with_order_df=join_df.select(cust_df["id"])

filtered_customer_df= cust_df.filter(~cust_df["id"].isin([row.id for row in cust_with_order_df.collect()]))

result_df=filtered_customer_df.orderBy("name")

result_df.show()


'''
Question 6:
Convert below query into pyspark code:

with result as (
select distinct caller_id, count(caller_id)
from calls c inner join users u
on c.caller_id=u.user_id
group by caller_id
having count(caller_id)>=3)
select count(*) from result;
'''

calls_df=spark.read.format("csv").option("inferSchema", True).option("header", True).load("calls1.csv")
users_df=spark.read.format("csv").option("inferSchema", True).option("header", True).load("users1.csv")

join_df=calls_df.join(users_df, calls_df.caller_id==users_df.user_id, "inner")

group_df=join_df.groupBy("caller_id").count()

filtered_df=group_df.filter(col("count")>=3)

filtered_df.count()


'''
Question 7:
Convert below query into pyspark code:

select rentalHistory.room_id, 
count(rentalHistory.room_id) as 'no_of_bookings',
sum(rentalHistory.number_nights * rooms.price) as 'total_earnings' from rentalHistory
inner join rooms on rooms.room_id=rentalHistory.room_id
group by rentalHistory.room_id
order by no_of_bookings desc ;
'''

rooms_df=spark.read.format("csv").option("header", True).load("rooms.csv")
hist_df=spark.read.format("csv").option("header", True).load("hist.csv")

join_df=hist_df.join(rooms_df, hist_df.room_id==rooms_df.room_id, "inner")

result_df=join_df.groupBy(hist_df.room_id).agg(count(hist_df.room_id).alias("no_of_bookings"), 
                                         sum(hist_df.number_nights*rooms_df.price).alias("total_earnings")).orderBy(desc("no_of_bookings"))
result_df.show()


'''
Question 8:
convert below query into pyspark code:

select d.restaurant_name, sum(o.sales_amount) as sales from delivery_orders d
inner join order_value o
on 
d.delivery_id=o.delivery_id
and d.actual_delivery_time is not null
group by  d.restaurant_name
order by sales desc
limit 2;
'''

delivery_df=spark.read.format("csv").option("header", True).load("delivery.csv")
order_df=spark.read.format("csv").option("header", True).load("order.csv")

result_df=delivery_df.join(order_df, delivery_df.delivery_id==order_df.delivery_id, "inner")\
.filter(delivery_df.actual_delivery_time.isNotNull())\
.groupBy(order_df.restaurant_name)\
.agg(sum(delivery_df.sales_amount).alias('sales'))\
.orderBy(desc('sales')).limit(2)

result_df.show()

'''
Question 9:
convert below query into pyspark code:
select driver_id, round(avg(order_total),3) as avg_total
from delivery_details
where datediff(minute,delivered_to_consumer_datetime,driver_at_restaurant_datetime )<=45
group by driver_id
having round(avg(order_total),3)>30
order by avg_total desc;
'''

result_df=df1.withColumn("time_diff", (unix_timestamp("delivered_to_consumer_datetime")-unix_timestamp("driver_at_restaurant_datetime"))/60)\
.filter(col("time_diff")<=45)\
.groupBy(col("driver_id"))\
.agg(round(avg(col("order_total")),3))\
.alias("avg_total")\
.filter(col("avg_total")>30)\
.orderBy(desc(col("avg_total")))\
.show()



'''
Question 10:
convert below query into pyspark code:
select shipment_id, weight
from 
(select shipment_id, weight, dense_rank() over(order by weight desc) as ranking from shipments) as a
where ranking=3

'''
df=spark.read.format("csv").option("header", True).load("shipments.csv")
window=Window.orderBy(col("weight").desc())
df1=df.withColumn("ranking", dense_rank().over(window))
df2=df1.filter(df1.ranking==3)
df1.show()



'''
Question 11:
convert below query to pyspark code:

with cte as (
select *, rank() over (partition by company order by year) as rank_by_year,
rank() over (partition by company order by revenue) as rank_by_revenue,
(cast(rank() over (partition by company order by year) as signed)
 - cast(rank() over (partition by company order by revenue)as signed)) as diff
from company_revenue
)

select company from cte
group by company
having count(distinct diff)=1 and avg(diff)=0;
'''

company_df=spark.read.format('csv').option('header', True).load("company_revenue.csv")

window=Window.partitionBy(col("company")).orderBy(col("year"))
window2=Window.partitionBy(col("company")).orderBy(col("revenue"))

result_df=company_df.withColumn("rank_by_year", rank().over(window))\
.withColumn("rank_by_revenue", rank().over(window2))\
.withColumn("diff", rank().over(window)- rank().over(window2))\

final_df=result_df.groupBy(col("company"))\
.agg(countDistinct("diff").alias("distinct_diff_count"), avg("diff").alias("avg_diff"))\
.filter((col("distinct_diff_count")==1) & (col("avg_diff")==0))

final_df.show()



'''
Question 12:
convert below query into pyspark code:

with cte as (
select *,
row_number() over (partition by subj_1, subj_2 order by marks) as ranks
from my_table1
)

select id, subj_1, subj_2, marks from cte where ranks<=1;

'''

df=spark.read.format("csv").option("header", True).load("my_table1.csv")
window=Window.partitionBy("subj_1", "subj_2").orderBy("marks")

result_df=df.withColumn("ranks", row_number().over(window))

final_df=result_df.filter(col("ranks")<=1)

result_df.show()

final_df.show()



'''
Question 13:
convert below query into pyspark code:

 with cte as(
 select sname, marks, row_number() over (partition by sname order by marks desc) as result
 from students1
)

select sname, sum(marks) as total from cte
where result<=2
group by sname

'''

df=spark.read.format("csv").option("header", True).load("students_1.csv")

window=Window.partitionBy("sname").orderBy(desc("marks"))

result_df=df.withColumn("results", row_number().over(window))

final_df=result_df.filter(result_df.results<=2)\
.groupBy("sname")\
.agg(sum(result_df.marks).alias("total"))

final_df.show()



'''
Question 14:
convert below query into pyspark code:

with cte as(
select *, row_number() over(partition by subjects order by marks) as result
from student_marks
)
select student_id, student_name, subjects, marks from cte
where result<=2;
'''

df=spark.read.format("csv").option("header", True).load("student_marks.csv")

#df.show()
window=Window.partitionBy("subjects").orderBy("marks")

result_df=df.withColumn("result", row_number().over(window))

#result_df.show()

final_df=result_df.filter(col("result")<=2).select("student_id","student_name","subjects","marks")

final_df.show()



'''
Question 15:
Calculate average marks, count of students for each subject

'''

df=spark.read.format("csv").option("header", True).load("student_marks.csv")

result_df=df.groupBy("subjects")\
.agg(avg("marks").alias("avg_marks"), count("student_name").alias("count"))\

result_df.show()



'''
Question 16:
Calculate Running total

'''

df=spark.read.format("csv").option("header", True).load("student_marks.csv")

window=Window.partitionBy("subjects").orderBy("student_id")

result_df=df.withColumn("result", sum(col("marks")).over(window))

result_df.show()



'''
Question 17:
To calculate the percentage of total salary that each employee contributes to their respective department.
'''

df=spark.read.format("csv").option("header", True).load("student_marks.csv")

#df.show()

df1=df.groupBy("subjects").agg(sum("marks").alias("total_marks"))

#df1.show()

join_df=df.join(df1, df.subjects==df1.subjects)

join_df.show()

result_df=join_df.withColumn("percentage", round((col("marks")/col("total_marks"))*100, 2))

result_df.show()



'''
Question 18:
Calculate Running average

'''

df=spark.read.format("csv").option("header", True).load("student_marks.csv")

window=Window.partitionBy("subjects").orderBy("student_id")

result_df=df.withColumn("average", avg(col("marks")).over(window))

result_df.show()



'''
Question 19:
convert below query into pyspark code:

with cte as (
select  product_category, avg(price) as average, max(price) as maximum
from products2
group by product_category
)

select a.*, c.* from products2 a join cte c 
on a.product_category=c.product_category
and a.price>c.average
and a.price<c.maximum;

'''

df=spark.read.format("csv").option("header", True).load("products2.csv")

window=Window.partitionBy("product_category")
result_df=df.withColumn("average", avg(col("price")).over(window)).withColumn("maximum", max(col("price")).over(window))

result_df=result_df.withColumnRenamed("price", "cost")
join_df=df.join(result_df, df.product_category==result_df.product_category ,"inner")\
.filter((df.price>result_df.average) & (df.price<result_df.maximum))

join_df.show()


'''
Question 20:
convert below query into pyspark code:

with cte as (
select *, datediff(day, order_date, date()) as diff
from orders4
)

select customer_id from cte where diff>7 and diff<=30;
'''

df=spark.read.format("csv").option("header", True).load("orders4.csv")

result_df=df.withColumn("diff", datediff(current_date(), col("order_date")))

final_df=result_df.filter((col("diff")>7) & (col("diff")<=30))

final_df.show()



'''
Question 21:
convert below query into pyspark code:

with cte as (
select *, avg(salary) over (partition by dept_id) as average from employee1
)

select emp_id , dept_id, name , salary, average  from cte
where salary>average
'''

df=spark.read.format("csv").option("header", True).load("employee1.csv")
window=Window.partitionBy("dept_id")

result_df=df.withColumn("average", avg("salary").over(window))

final_df=result_df.filter(col("salary")>col("average")).select("emp_id", "name", "dept_id", "salary")

final_df.show()



'''
Question 22:

data=[("1", "abc", "5647 7463 7678 8625"), ("2", "xyz","7987 7867 7862 7353")]

column_names=["id", "name", "card"]

df=spark.createDataFrame(data, column_names)

Given above example, mask name column values as from the list 
for example, mask "5647 7463 7678 8625" with "**** **** **** 8625" and "7987 7867 7862 7353" with "**** **** **** 8625"
'''

result_df=df.withColumn("name", regexp_replace(col("name"), r"\d{4} \d{4} \d{4}", "**** **** ***"))

result_df.show()



'''
Question 23:

How to check Spark UI through Jupyter notebook:
'''

import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast

spark=SparkSession.builder.getOrCreate()

spark.sparkContext.uiWebUrl    #with the help of this command, we can go to spark UI to monitor 



'''
Question 24:
Given a large dataset that doesn’t fit in memory, how would you convert a Pandas DataFrame to a PySpark DataFrame for scalable processing?
'''

from pyspark.sql import SparkSession
import pandas as pd

spark=SparkSession.builder.getOrCreate()

chunksize=10**6   #here we have specified chunksize as 1 million per chunk

df=pd.read_csv("large_dataset.csv", chunksize=chunksize)

spark_df=spark.createDataFrame(df)

spark_df.show()



'''
Question 25:
Given the CustomerOrders table, 
Determine the percentage of unpaid orders per region, calculated as the number of unpaid orders divided by the total number of orders in that region.
'''

df=spark.read.format("csv").option("header", True).load("customerOrders.csv")
df1=df.filter(col("PaymentStatus")=="Unpaid").groupBy("Region").count()

df2=df.groupBy("Region").agg(count("OrderID").alias("cnt"))

final_df=df1.join(df2, df1.Region==df2.Region, "inner").withColumn("Percentage", round((df1["count"]/df2["cnt"])*100,2))

result_df=final_df.orderBy(col("Percentage").desc())

result_df.select(df1["Region"], "Percentage").show()



'''
Question 26:
Convert below query into pyspark code:

with cte as (
select *, row_number() over (partition by state order by population desc) as max_pop,
row_number() over (partition by state order by population) as min_pop from city_population
)

select c.* from cte c where c.max_pop=1 or c.min_pop=1
'''

df=spark.read.format("csv").option("header", True).load("city_population.csv")

window1=Window.partitionBy("state").orderBy(desc("population"))

window2=Window.partitionBy("state").orderBy("population")

result_df=df.withColumn("max_pop", row_number().over(window1)).withColumn("min_pop", row_number().over(window2))

final_df=result_df.filter((result_df.max_pop==1) | (result_df.min_pop==1))

final_df.show()



'''
Question 27:
Convert below query into pyspark code:

with cte as (
select month(orderdate) as mn, avg(sales_amount) as avg_sales from salesdata group by mn
)

select c.*, a.productid from cte c join salesdata a on c.mn=month(a.orderdate) and a.sales_amount>c.avg_sales
'''

df=spark.read.format("csv").option("header", True).load("salesdata.csv")

window=Window.partitionBy(month("orderdate"))

df1=df.withColumn("avg_sales", avg("sales_amount").over(window))

result_df=df1.filter(col("sales_amount")>col("avg_sales"))

result_df.show()



'''
Question 28:
Convert below query into pyspark code:

with cte as (
select *, instr(email, '@') as position from new_customer
)
select count(customer_id) as cnt, substr(email, position, length(email)-1) as domain 
 from cte group by substr(email, position, length(email)-1) order by cnt desc

'''

df=spark.read.format("csv").option("header", True).load("new_customer.csv")

df1=df.withColumn("position", instr(df.email, "@"))

df2=df1.withColumn("domain", substr(df.email, df1.position, length(df.email)-1))

final_df=df2.groupBy("domain").agg(count(df.customer_id).alias("cnt")).orderBy(desc(count(df.customer_id)))

final_df.show()



'''
Question 29:
Convert below query into pyspark code:

with cte as(
select *, quantity*unit_price as sales, row_number() over (partition by region order by (quantity*unit_price) desc) as result,
sum(quantity*unit_price) over (partition by region) as total_sales from saledata
)

select *, (sales/total_sales)*100 as percentage from cte where result=1 and total_sales>500

'''

df=spark.read.format("csv").option("header", True).load("sale_data.csv")
window=Window.partitionBy("region").orderBy(desc(col("quantity")*col("unit_price")))

window2=Window.partitionBy("region")
result_df=df.withColumn("sales", col("quantity")*col("unit_price"))

df1=result_df.withColumn("result", row_number().over(window))\
.withColumn("total_sales", sum("sales").over(window2))\
.withColumn("percentage", (col("sales")/col("total_sales"))*100)\
.filter((col("result")==1) & (col("total_sales")>500))

df1.show()



'''
Question 30:
Convert below query into pyspark code:

select 
case when rating between 8.0 and 8.9 then "8.0-8.9" 
when rating between 6.0 and 6.9 then "6.0-6.9"
when rating between 7.0 and 7.9 then "7.0-7.9"
when rating between 9.0 and 9.9 then "9.0-9.9"
when rating =10.0 then "perfect 10"
else  "not recommended"
end as ranges,
count(*) as counts
from MovieData
group by ranges;

'''

df=spark.read.format("csv").option("header", True).load("movie_data.csv")

df_temp=df.withColumn("rating_range", when((df.rating>=8.0) & (df.rating<=8.9), "8.0-8.9")\
                     .when((df.rating>=9.0) & (df.rating<=9.9), "9.0-9.9")\
                     .when((df.rating>=7.0) & (df.rating<=7.9), "7.0-7.9")\
                     .when((df.rating>=6.0) & (df.rating<=6.9), "6.0-6.9")\
                     .when(df.rating==10.0, "perfect_10")\
                     .otherwise("not recommended"))

result_df=df_temp.groupBy("rating_range").count()

result_df.show()



'''
Question 31:
Convert below query into pyspark code:

with cte as (select *, month(dates) as mn from revenue_table)
select *, sum(revenue) over(partition by mn order by dates desc) as total from cte 
'''

r_df=spark.read.format("csv").option("header", True).load("revenue_table.csv")

df=r_df.withColumn("mn", month("dates"))

window=Window.partitionBy("mn").orderBy(desc("dates"))

result_df=df.withColumn("total", sum("revenue").over(window))

result_df.show()



'''
Question 32:
Convert below query into pyspark code:

with cte as (
select * from flights where departure_date=actual_departure_date
)
select count(c.flight_id ) as on_time_departures,  count(*) as total_flights from flights left join cte c
on flights.flight_id=c.flight_id
'''

df=spark.read.format("csv").option("header", True).load("flights.csv")

window=Window.orderBy("flight_id")

filter_df=df.filter(df.departure_date==df.actual_departure_date)

join_df=df.join(filter_df, df.flight_id==filter_df.flight_id, "left")

result_df=df.withColumn("total_flights", count(df.flight_id).over(window))

final_df=filter_df.withColumn("on_time_departure", count(filter_df.flight_id).over(window))

result_df.orderBy(desc("total_flights")).show(1)

final_df.orderBy(desc("on_time_departure")).show(1)

final_join=result_df.join(final_df, result_df.flight_id==final_df.flight_id, "left")

final_join.agg(max("total_flights").alias("total_flights"), max("on_time_departure").alias("on_time_departure")).show()



'''
Question 33:
Convert below query into pyspark code:

select city, 
count(*) as Total_Count,
count(case when gender='M' then 1 end) as Male_count,
count(case when gender='F' then 1 end) as Female_count,
sum(case when gender='M' then salary end) as Total_Male_salary,
sum(case when gender='F' then salary end) as Total_Female_salary
from cities group by city
'''

df=spark.read.format("csv").option("header", True).load("cities.csv")

result_df=df.groupBy("city").agg(count("city").alias("total_count"), 
                                 count(when(df.gender=="M", 1)).alias("Male_Count"),
                                 count(when(df.gender=="F", 1)).alias("Female_count"),
                                 sum(when(df.gender=="M", df.salary)).alias("Total_Male_Salary"),
                                 sum(when(df.gender=="F", df.salary)).alias("Total_female_Salary"))
result_df.show()



'''
Question 34:

You are given with the dataset below, get the RESULTANT dataset using PYSPARK:

Input dataset:
+----------+------------+
|Department|EmployeeName|
+----------+------------+
|    IT|    John|
|    IT|    Jane|
|    IT|   Robert|
|    IT|    Emily|
|    HR|    Mike|
|    HR|    Alice|
|    HR|   Sophia|
|    HR|   Daniel|
|  Finance|   Olivia|
|  Finance|    Liam|
| Marketing|    Emma|
| Marketing|    Noah|
+----------+------------+

Resultant dataset:
+---------+---------------------------------+
|     dept|              e_list         |cnt|
+---------+-----------------------------+---+
|       HR|[Mike, Alice, Sophia, Daniel]|  4|
|  Finance|      [Olivia, Liam]         |  2|
|Marketing|        [Emma, Noah]         |  2|
|       IT|[John, Jane, Robert, Emily]  |  4|
+---------+-----------------------------+---+
'''

spark=SparkSession.builder.getOrCreate()

df=spark.read.format("csv").option("header", True).load("emp_1.csv")

df.show()

result_df=df.groupBy("dept").agg(collect_list("name").alias("e_list"),count("dept").alias("cnt"))

result_df.show()



'''
Question 35:
You are given with the dataset below, Identify rows containing non-numeric values in the "Quantity" column, if any using pyspark.

Input data:
+-----------+--------------+---------+------+
|productCode|      Quantity|unitPrice|custId|
+-----------+--------------+---------+------+
|       P001|             5|       20|  C001|
|       P002|             &|     15.5|  C002|
|       P003|          10-n|     5.99|  C003|
|       P004|             2|    18.98|  C001|
|       P005|         eight|    12.75|  C002|
|       P006|seven and half|   118.75|  C006|
+-----------+--------------+---------+------+
'''

df=spark.read.format("csv").option("header", True).load("store.csv")

df.show()

result_df=df.filter(col("Quantity").rlike('[^0-9.]'))

result_df.show()



'''
Question 36:
Convert null values with specific values in below data:
+----+--------+----+
|  id|    name| age|
+----+--------+----+
|   1|Van Dijk|  23|
|   2|    NULL|  32|
|   3|   Messi|NULL|
|   4|    NULL|NULL|
|   5|    Kaka|NULL|
|   6| Ronaldo|  35|
|   7|    NULL|  28|
|   8|   Messi|NULL|
|NULL|  Neymar|  29|
|  10|  Mbappe|  22|
+----+--------+----+
'''

df=spark.read.format("csv").option("header", True).load("person.csv")

result_df=df.na.fill({"id":5, "name":"messi", "age":28})

result_df.show()



'''
Question 37:
Convert below query into pyspark code:

with cte as (
select *, row_number() over(partition by id order by marks desc) as n from students_new  
),

cte2 as (
select * from cte where n<=3
)
select id, avg(marks) from cte2 group by id
'''

df=spark.read.format("csv").option("header", True).load("students_new.csv")

window=Window.partitionBy("id").orderBy(desc("marks"))

window2=Window.partitionBy("id")

df1=df.withColumn("n", row_number().over(window))

df2=df1.filter(df1.n<=3)

result_df=df2.withColumn("average", avg("marks").over(window2))

result_df.select("id", "average").distinct().show()



'''
Question 38:
Convert below query into pyspark code:

select a.student_id from StudentSubjects a join StudentSubjects b on
a.subject="Chemistry" and b.subject="Physics" and
a.student_id=b.student_id and
a.marks=b.marks;
'''

df1=spark.read.format("csv").option("header", True).load("StudentSubjects.csv")

df2=spark.read.format("csv").option("header", True).load("StudentSubjects.csv")

r_df=df1.join(df2, (df1.student_id==df2.student_id) & (df1.marks==df2.marks), "inner").filter((df1.subject=="Chemistry") & (df2.subject=="Physics")).select(df1.student_id)

r_df.show()



'''
Qustion 39:
Convert below query into pyspark code:

with cte as (
select CUSTOMER_ID, SUM(REVENUE) as total_revenue from CustomerPurchases group by CUSTOMER_ID	
),
cte2 as (
select CUSTOMER_ID, REVENUE  from CustomerPurchases where PRODUCT="Photoshop"
)
select a.CUSTOMER_ID, a.total_revenue-b.REVENUE as final_revenue from cte a join cte2 b on a.CUSTOMER_ID=b.CUSTOMER_ID
'''

df=spark.read.format("csv").option("header", True).load("CustomerPurchases.csv")
window=Window.partitionBy("cust_id")

df1=df.withColumn("total_revenue", sum("revenue").over(window)).withColumnRenamed("cust_id", "customer_id")

df2=df1.select("customer_id", "total_revenue").distinct()

df3=df.filter(col("product")=="Photoshop")

join_df=df2.join(df3, df2.customer_id==df3.cust_id, "inner")

final_df=join_df.withColumn("final_revenue", join_df.total_revenue-join_df.revenue)

final_df.select("customer_id", "final_revenue").show()



'''
Question 40:
Convert below query into pyspark code:

with cte as (
select a.product_id, a.name, a.category_id, avg(b.rating) as average_rating, 
dense_rank() over (partition by a.category_id order by avg(b.rating) desc) as category_rank 
from Products a join Reviews b on a.product_id=b.product_id
group by a.product_id, a.name, a.category_id
)

select product_id, name, category_id, average_rating from cte where category_rank<=3;

'''

df1=spark.read.format("csv").option("header", True).load("product.csv")

temp_df=df1.withColumnRenamed("product_id", "pid")

df2=spark.read.format("csv").option("header", True).load("reviews.csv")

join_df=temp_df.join(df2, temp_df.pid==df2.product_id, "inner")

result_df=join_df.groupBy("pid", "name ", "category").agg(avg("rating").alias("average_rating"))

window=Window.partitionBy("category").orderBy(desc("average_rating"))

final_df=result_df.withColumn("category_rank", dense_rank().over(window))

final=final_df.filter(col("category_rank")<=3)

final.show()



'''
Question 41:
Convert below query into pyspark code:

with cte as(
select *, row_number() over (partition by id order by order_date) as result from orders
),
cte2 as (
select *, lead(order_date) over(partition by id order by result) as next_date from cte
)
select id from cte2 where next_date-order_date=1;
'''

df=spark.read.format("csv").option("header", True).load("orders1.csv")

window=Window.partitionBy("id").orderBy("order_date")
window2=Window.partitionBy("id").orderBy("result")

df1=df.withColumn("result", row_number().over(window))

df2=df1.withColumn("next_date", lead("order_date").over(window2))

final_df=df2.withColumn("diff", datediff(col("next_date"), col("order_date")))

final_df.filter(col("diff")==1).select("id").show()



'''
Question 42:
explode() in pyspark
'''

df=spark.read.format("csv").option("header", True).load("city.csv")

result_df=df.withColumn("locations", split("location", ","))

result_df=result_df.withColumn("results", explode("locations"))
result_df.show()



'''
Question 43:
bucketBy() in pyspark
'''

df=spark.read.format("csv").option("header", True).load("CustomerPurchases.csv")

df.show()

df3=df.write.bucketBy(2, "product").saveAsTable("bucketed_table1")

result_df=spark.read.table("bucketed_table1")

result_df.show()



'''
Question 44:
Convert below query into pyspark code:

with cte as(
select *, row_number() over (partition by city) as city_count from city_gender
),
cte2 as (
select city, gender, max(city_count) over (partition by city) as max_count from cte
),
cte3 as (
select *, count(gender) over (partition by city) as male_count from cte2 where gender='M'
),
cte4 as(
select *, count(gender) over (partition by city) as female_count from cte2 where gender='F'
)
select distinct a.city, a.max_count, a.male_count, b.female_count from cte3 a left join cte4 b on a.city=b.city;
'''

df=spark.read.format("csv").option("header", True).load("city_gender.csv")

window=Window.partitionBy("city").orderBy("city")

result_df=df.withColumn("city_count", row_number().over(window))

result_df=result_df.withColumn("max_count", max("city_count").over(window))

df1=result_df.withColumn("male_count", count(when(col("gender")=='M', 1)\
                                                .otherwise(None)).over(window))

df2=result_df.withColumn("female_count", count(when(col("gender")=='F', 1)\
                                                  .otherwise(None)).over(window))

join_df=df1.join(df2, df1.city==df2.city, "left")

join_df.select(df1.city, df1.max_count, df1.male_count, df2.female_count).distinct().show()


'''
Question 45:
split() function in pyspark:
'''

df=spark.read.format("csv").option("header", True).load("marks.csv")

result_df=df.withColumn("marks", split("marks", "\\|"))

result_df=result_df.withColumn("physics", result_df.marks[0])\
.withColumn("chemistry", result_df.marks[1])\
.withColumn("maths", result_df.marks[2])

result_df.show()



'''
Question 46:
For below data, replace non-numeric values with empty values and calculate age of customers by calculating difference between registered date & current date:
+-----------+---------------+--------------------+-------------+----------+
|customer_id|           name|               email|        phone|  reg_date|
+-----------+---------------+--------------------+-------------+----------+
|          1|       John Doe|  john.doe@gmail.com| 123-456-7890|2022-01-15|
|          2|     Jane Smith|jane.smith@hotmai...|(987)654-3210|2021-11-30|
|          3|      Alice Lee| alice.lee@yahoo.com|     555-5555|2023-03-10|
|          4|      Bob Brown| bob.brown@gmail.com|         NULL|2022-05-20|
|          5|Charlie Johnson|charlie.johnson@g...|     789-0123|2021-08-12|
|          6|      Eva Davis| eva.davis@yahoo.com|(123)456-7890|2022-09-18|
|          7|   Frank Wilson|frank.wilson@hotm...|     111-2222|2023-01-05|
+-----------+---------------+--------------------+-------------+----------+
'''

df=spark.read.format("csv").option("header", True).load("customer.csv")

result_df=df.withColumn("phone", regexp_replace(col("phone"), '[^0-9]', ""))  #replaced non-numeric values with empty space

result_df=result_df.withColumn("reg_date", to_date(result_df.reg_date, "yyyy-mm-dd"))

result_df=result_df.withColumnRenamed("name", "Full name")\
                    .withColumn("current_date", current_date())\
                    .withColumn("age", (current_date()-col("reg_date")).cast(IntegerType()))

result_df.show()



'''
Question 45:
Remove duplicates from list
lists=[1,2,3,4,5,5,2]
'''

df = spark.read.format("csv").option("header", True).load("lists.csv")

window=Window.partitionBy("result").orderBy("result")

df=df.withColumn("splits", split("lists", ","))

df=df.withColumn("result", explode("splits"))

df=df.withColumn("ranks", row_number().over(window))

df=df.filter(col("ranks")==1)

df=df.select("result")

result_df=df.agg(collect_list("result").alias("final"))

result_df.show()



'''
Question 46:
1.to get current date
2.no. of days difference between current date and joining date
3.no. of months difference between current date and joining date
'''

df=spark.read.format("csv").option("header", True).load("items.csv")
df.show()
df1=df.withColumn("current", current_date())  #to get current date
df1.show()
df2=df1.withColumn("joining", lit("2024-12-01").cast("date"))
df2.show()
df3=df2.withColumn("diff", datediff(col("current"), col("joining")))  #no. of days difference between current date and joining date
df3.show()
df4=df3.withColumn("month_diff", months_between(col("current"), col("joining")))   #no. of months difference between current date and joining date
df4.show()



'''
Question 47:
Swapping items with lead() and lag()
'''

window=Window.orderBy("order_id")
df=spark.read.format("csv").option("header", True).load("items_order.csv")
df2=df.withColumn("correct_ordered_items", when(col("order_id")%2!=0, coalesce(lead("item").over(window), col("item"))).otherwise(lag("item").over(window)))  
#It takes the next row's "item" value (or the current "item" value if there's no next row).
df2.show()


'''
Question 48:

'''

+-------+------------+-------------+
|   City|grouped_city|activity_rank|
+-------+------------+-------------+
| Mumbai|           2|            1|
|Newyork|           2|            2|
| Nashik|           1|            3|
+-------+------------+-------------+



AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `user_id` cannot be resolved. Did you mean one of the following? [`City`, `Id`, `State`, `County`, `Zipcode`].;
'Aggregate ['user_id], ['user_id, sum('listen_duration) AS total_listening#93, 'count(distinct 'song_id) AS distinct_songs#94]
+- Relation [Id#17,Institution_Name#18,Branch_Name#19,Branch_Number#20,City#21,County#22,State#23,Zipcode#24,2010_Deposits#25,2011_Deposits#26,2012_Deposits#27,2013_Deposits#28,2014_Deposits#29,2015_Deposits#30,2016_Deposits#31] csv


In [5]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast

spark=SparkSession.builder.getOrCreate()

df=spark.read.format("csv").option("header", True).load("employees.csv")
#df.show()

result_df=df.agg(avg("age").alias("average_age"), avg("salary").alias("average_salary")).collect()[0]
avg_df=result_df.average_age
avg_sal=result_df.average_salary

result_df=df.na.fill({"age": avg_df, "salary": avg_sal})

result_df.show()


+------+-------+----+------+
|emp_id|   name| age|salary|
+------+-------+----+------+
|     1|  alice|  25|  5000|
|     2|    bob|  30|  6000|
|     3|charlie|30.0|  5500|
|     4|  david|  35|5500.0|
|     5|    eve|30.0|5500.0|
+------+-------+----+------+



In [47]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import instr
from pyspark.sql.functions import expr


spark=SparkSession.builder.getOrCreate()

window=Window.orderBy(desc("total_sales"))

window2=Window.partitionBy("city")

df=spark.read.format("csv").option("header", True).load("trades.csv")

df2=spark.read.format("csv").option("header", True).load("users.csv")

df=df.withColumn("total_sales", col("quantity")*col("price"))

df=df.withColumn("ranks", row_number().over(window))

df=df.filter(col("status")=='Completed')

df2=df2.withColumn("city_count", count("city").over(window2))

#df2.show()

join_df=df.join(df2, df.stock_id==df2.user_id, "inner")

join_df.show()

join_df.groupBy("city").count()


+--------+--------+-----+--------+---------+-------------------+-----------+-----+-------+-------------+--------------------+-------------------+----------+
|trade_id|stock_id|price|quantity|   status|         trade_date|total_sales|ranks|user_id|         city|               email|          join_date|city_count|
+--------+--------+-----+--------+---------+-------------------+-----------+-----+-------+-------------+--------------------+-------------------+----------+
|  100264|     148|  4.8|      40|Completed|2022-08-26 12:00:00|      192.0|    1|    148|       Boston|sailor9820@gmail.com|2021-08-20 12:00:00|         1|
|  100259|     148|  5.1|      35|Completed|2022-08-25 12:00:00|      178.5|    2|    148|       Boston|sailor9820@gmail.com|2021-08-20 12:00:00|         1|
|  100305|     300|   10|      15|Completed|2022-09-05 12:00:00|      150.0|    3|    300|San Francisco|houstoncowboy1122...|2022-06-30 12:00:00|         3|
|  100400|     178|  9.9|      15|Completed|2022-09-09 12:

DataFrame[city: string, count: bigint]

In [48]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import instr
from pyspark.sql.functions import expr


spark=SparkSession.builder.getOrCreate()

df=spark.read.format("csv").option("header", True).load("temp_3.csv")

df1=df.withColumn("result", regexp_replace("name", "[^0-9]", ""))

df1=df1.withColumn("result", df1.result.cast("Integer"))

df1.show()

+----------+--------+
|      name|  result|
+----------+--------+
|20/04/1963|20041963|
|21/04/1964|21041964|
|23/04/1962|23041962|
+----------+--------+



In [13]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import explode

spark=SparkSession.builder.getOrCreate()
df = spark.read.format("csv").option("header", True).load("lists.csv")

window=Window.partitionBy("result").orderBy("result")

df=df.withColumn("splits", split("lists", ","))

df=df.withColumn("result", explode("splits"))

df=df.withColumn("ranks", row_number().over(window))

df=df.filter(col("ranks")==1)
df.show()
df=df.select("result")

result_df=df.agg(collect_list("result").alias("final"))
result_df.show()

df=df.agg(max("result").alias("max"), min("result").alias("min"))
df.show()

+---------------+--------------------+------+-----+
|          lists|              splits|result|ranks|
+---------------+--------------------+------+-----+
|1,2,3,4,5,5,6,2|[1, 2, 3, 4, 5, 5...|     1|    1|
|1,2,3,4,5,5,6,2|[1, 2, 3, 4, 5, 5...|     2|    1|
|1,2,3,4,5,5,6,2|[1, 2, 3, 4, 5, 5...|     3|    1|
|1,2,3,4,5,5,6,2|[1, 2, 3, 4, 5, 5...|     4|    1|
|1,2,3,4,5,5,6,2|[1, 2, 3, 4, 5, 5...|     5|    1|
|1,2,3,4,5,5,6,2|[1, 2, 3, 4, 5, 5...|     6|    1|
+---------------+--------------------+------+-----+

+------------------+
|             final|
+------------------+
|[1, 2, 3, 4, 5, 6]|
+------------------+

+---+---+
|max|min|
+---+---+
|  6|  1|
+---+---+



In [17]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import instr
from pyspark.sql.functions import expr


spark=SparkSession.builder.getOrCreate()
df=spark.read.format("csv").option("header", True).load("items.csv")
#df.show()
df1=df.withColumn("current", current_date())  #to get current date
#df1.show()
df2=df1.withColumn("joining", lit("2024-12-01").cast("date"))
#df2.show()
df3=df2.filter(col("joining").between("2024-11-01", "2024-11-16"))
df3.show()

+-----+-----+-------+-------+
|items|price|current|joining|
+-----+-----+-------+-------+
+-----+-----+-------+-------+



In [35]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import instr
from pyspark.sql.functions import expr
from pyspark.sql.functions import col, array_contains


spark=SparkSession.builder.getOrCreate()

df1=spark.read.format("csv").option("header", True).load("cust.csv")

df2=spark.read.format("csv").option("header", True).load("ord.csv")

window=Window.partitionBy("customer_id").orderBy("customer_id")

df2=df2.withColumn("ranks", row_number().over(window))

join_df=df1.join(df2, df1.customer_id==df2.customer_id, "inner").select(df1.customer_id, df1.customer_name, df2.product_name)
join_df.show()

join_df=join_df.groupBy("customer_name", "customer_id").agg(collect_list("product_name").alias("list"))
join_df.show()

join_df=join_df.filter((~array_contains(col("list"), 'C')) & (array_contains(col("list"), 'A')) & (array_contains(col("list"), 'B')) & (array_contains(col("list"), 'D')))
join_df.select("customer_name", "customer_id").show()

+-----------+-------------+------------+
|customer_id|customer_name|product_name|
+-----------+-------------+------------+
|          1|       Daniel|           A|
|          1|       Daniel|           B|
|          1|       Daniel|           D|
|          1|       Daniel|           C|
|          2|        Diana|           A|
|          3|    Elizabeth|           A|
|          3|    Elizabeth|           B|
|          3|    Elizabeth|           D|
|          4|         John|           C|
+-----------+-------------+------------+

+-------------+-----------+------------+
|customer_name|customer_id|        list|
+-------------+-----------+------------+
|         John|          4|         [C]|
|       Daniel|          1|[A, B, D, C]|
|        Diana|          2|         [A]|
|    Elizabeth|          3|   [A, B, D]|
+-------------+-----------+------------+

+-------------+-----------+
|customer_name|customer_id|
+-------------+-----------+
|    Elizabeth|          3|
+-------------+----------

In [20]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import instr
from pyspark.sql.functions import expr
from pyspark.sql.functions import col, array_contains


spark=SparkSession.builder.getOrCreate()

df=spark.read.format("csv").option("header", True).load("tran.csv")

df=df.withColumn("tran_dt", col("tran_dt").cast("timestamp"))

window=Window.partitionBy("user_id").orderBy("tran_dt")

df=df.withColumn("next_dt", lead("tran_dt").over(window))

df=df.withColumn("diff", datediff("next_dt", "tran_dt"))

window2=Window.partitionBy("user_id")

df=df.withColumn("cnt", count("trans_is").over(window2))\
    .withColumn("total", sum("diff").over(window2))

df=df.withColumn("avg", col("total")/col("cnt"))

result_df=df.select("trans_is", "user_id", "tran_amt", "avg")
result_df.show()

result_df=result_df.filter()

+--------+-------+--------+---+
|trans_is|user_id|tran_amt|avg|
+--------+-------+--------+---+
|       1|    101|     500|2.0|
|       3|    101|     300|2.0|
|       7|    101|     200|2.0|
|       2|    102|     200|1.5|
|       5|    102|     400|1.5|
|       4|    103|     100|1.0|
|       6|    103|     600|1.0|
+--------+-------+--------+---+



TypeError: DataFrame.filter() missing 1 required positional argument: 'condition'

In [21]:
import pyspark, os, sys
from pyspark.sql import *
from pyspark import SparkConf,SparkContext
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import types
from pyspark import StorageLevel
from pyspark.sql.functions import broadcast
from pyspark.sql.functions import instr
from pyspark.sql.functions import expr
from pyspark.sql.functions import col
from pyspark.sql.functions import regexp_extract
from pyspark.sql.functions import posexplode
from pyspark.sql.functions import spark_partition_id
from pyspark.sql.functions import substring
from pyspark.sql.functions import array_contains


spark=SparkSession.builder.getOrCreate()

old_df=spark.read.format("csv").option("header", True).load("sample_scd.csv")

new_df=spark.read.format("csv").option("header", True).load("sample_scd2.csv")

old_df=old_df.withColumn("start_date", col("start_date").cast("date"))\
            .withColumn("end_date", col("end_date").cast("date"))\
            .withColumn("salary", col("salary").cast("Integer"))

new_df=new_df.withColumn("salary", col("salary").cast("Integer"))

new_df=new_df.withColumnRenamed("id", "id_new")\
            .withColumnRenamed("name", "name_new")\
            .withColumnRenamed("salary", "salary_new")

old_df=old_df.withColumn("hash_old", hash("id", "name", "salary"))
new_df=new_df.withColumn("hash_new", hash("id_new", "name_new", "salary_new"))

join_df=old_df.join(new_df, old_df.id==new_df.id_new, "full")
#join_df.show()

#unchanged
print("unchanged")
df1=join_df.filter(col("hash_new")==col("hash_old"))
df1=df1.select("id", "name", "salary", "start_date", "end_date")
df1.show()

#new
print("new")
df2=join_df.filter(col("hash_old").isNull())
df2=df2.select("id_new", "name_new", "salary_new")\
        .withColumn("start_date", current_date())\
        .withColumn("end_date", lit("2999-12-31").cast("date"))
df2.show()

#updated
print("updated")
df3=join_df.filter(col("hash_old")!=col("hash_new"))
df3_a=df3.select("id", "name", "salary", "start_date", "end_date")\
        .withColumn("end_date", current_date()-1)
df3_a.show()

df3_b=df3.select("id_new", "name_new", "salary_new")\
        .withColumn("start_date", current_date())\
        .withColumn("end_date", lit("2999-12-31").cast("date"))
df3_b.show()

print("final SCD2 result:")
final_df=df1.union(df2)\
            .union(df3_a)\
            .union(df3_b)
final_df.show()

unchanged
+---+----+------+----------+----------+
| id|name|salary|start_date|  end_date|
+---+----+------+----------+----------+
|  3|  cc|   300|2025-01-01|2999-12-31|
+---+----+------+----------+----------+

new
+------+--------+----------+----------+--------+
|id_new|name_new|salary_new|start_date|end_date|
+------+--------+----------+----------+--------+
+------+--------+----------+----------+--------+

updated
+---+----+------+----------+----------+
| id|name|salary|start_date|  end_date|
+---+----+------+----------+----------+
|  1|  aa|   100|2025-01-01|2025-05-29|
|  2|  bb|   200|2025-01-01|2025-05-29|
+---+----+------+----------+----------+

+------+--------+----------+----------+----------+
|id_new|name_new|salary_new|start_date|  end_date|
+------+--------+----------+----------+----------+
|     1|     aaa|       100|2025-05-30|2999-12-31|
|     2|     bbb|       200|2025-05-30|2999-12-31|
+------+--------+----------+----------+----------+

final SCD2 result:
+---+----+---